In [6]:
## 针对最朴素的PointNetv1的一个改进
#  改进一、Conv2d改成了Conv1d，这是为了用Batchnorm1d
#  改进二、读了GitHub上程序才发现每一层都加了batchnorm1d，遂加上，原文中在图注里讲了，没看到

#  待改进、两个Tnet即transform变换没有加入进去


import numpy as np
import torch
import torch.nn as nn

import math


class PointNet(nn.Module):
    def __init__(self,input_num1,input_num2,out_num):
        super(PointNet, self).__init__()
        
        self.num1 = input_num1
        self.block11 = nn.Conv1d(in_channels=input_num1,out_channels=64,kernel_size=1)
        self.block21 = nn.Conv1d(in_channels=64,out_channels=64,kernel_size=1)
    
        self.block31 = nn.Conv1d(in_channels=64,out_channels=128,kernel_size=1)
        self.block41 = nn.Conv1d(in_channels=128,out_channels=256,kernel_size=1)
    
        self.block12 = nn.Conv1d(in_channels=input_num2,out_channels=64,kernel_size=1)
        self.block22 = nn.Conv1d(in_channels=64,out_channels=64,kernel_size=1)
    
        self.block32 = nn.Conv1d(in_channels=64,out_channels=128,kernel_size=1)
        self.block42 = nn.Conv1d(in_channels=128,out_channels=256,kernel_size=1)
        
        self.block1 = nn.Conv1d(in_channels=input_num2,out_channels=64,kernel_size=1)
        self.block2 = nn.Conv1d(in_channels=64,out_channels=64,kernel_size=1)
    
        self.block3 = nn.Conv1d(in_channels=64,out_channels=128,kernel_size=1)
        self.block4 = nn.Conv1d(in_channels=128,out_channels=512,kernel_size=1)
    
    
        self.block5 = nn.Conv1d(in_channels=1216,out_channels=512,kernel_size=1)
        self.block6 = nn.Conv1d(in_channels=512,out_channels=256,kernel_size=1)
        self.block7 = nn.Conv1d(in_channels=256,out_channels=128,kernel_size=1)
    
        self.bct11 = nn.BatchNorm1d(64)
        self.bct21 = nn.BatchNorm1d(64)
        self.bct31 = nn.BatchNorm1d(128)
        self.bct41 = nn.BatchNorm1d(512)
        
        self.bct12 = nn.BatchNorm1d(64)
        self.bct22 = nn.BatchNorm1d(64)
        self.bct32 = nn.BatchNorm1d(128)
        self.bct42 = nn.BatchNorm1d(512)
        
        self.bct5 = nn.BatchNorm1d(512)
        self.bct6 = nn.BatchNorm1d(256)
        self.bct7 = nn.BatchNorm1d(128)
    
        self.block8 = nn.Conv1d(in_channels=128,out_channels=out_num,kernel_size=1)

    def forward(self, x):
        x1,x2 = x.split(self.num1,1)
        n = len(x)
        
        x = torch.relu(self.bct1(self.block1(x)))
        x = torch.relu(self.bct2(self.block2(x)))

        y = torch.relu(self.bct3(self.block3(x)))
        y = torch.relu(self.bct4(self.block4(y)))
        y = torch.max(y, 0, keepdim=True)[0]

        
        x1 = torch.relu(self.bct11(self.block11(x1)))
        x1 = torch.relu(self.bct21(self.block21(x1)))
        
        y1 = torch.relu(self.bct31(self.block31(x1)))
        y1 = torch.relu(self.bct41(self.block41(y1)))
        y1 = torch.max(y1, 0, keepdim=True)[0]
        
        
        x2 = torch.relu(self.bct12(self.block12(x2)))
        x2 = torch.relu(self.bct22(self.block22(x2)))
        
        y2 = torch.relu(self.bct32(self.block32(x2)))
        y2 = torch.relu(self.bct42(self.block42(y2)))
        y2 = torch.max(y2, 0, keepdim=True)[0]
        
        
        
        x = torch.cat((x,torch.cat((x1,x2),1)),1)
        y = torch.cat((y,torch.cat((y1,y2),1)),1)
        
        
        y = y.repeat(n,1,1)

        x = torch.cat((x,y),1)
        

        x = torch.relu(self.bct5(self.block5(x)))
        x = torch.relu(self.bct6(self.block6(x)))
        x = torch.relu(self.bct7(self.block7(x)))
        
        x = self.block8(x)
        
        return x


In [8]:
net1 = PointNet(2,1,1)

In [ ]:
from numba import cuda, float32
@cuda.jit
def compute_G2(G2_r, G2_i, x, c, k, const, ver_num):
    p, i, j = cuda.grid(3)
    if p < G2_r.shape[0] and i < G2_r.shape[1] and j < G2_r.shape[2]:
        if not (i in ver_num):
            p_n = 0.0
            for d in range(2):
                r = x[p,i,d]-x[p,:,d]
                p_n += c[p,i,d]*r
            d = 0.0
            for d in range(2):
                d += (x[p,i,d]-x[p,j,d])**2
            d = math.sqrt(d)
            h1 = hankel1(1,k*d)
            G2_r[p,i,j] = -( h1.imag/4*(p_n)/d*k )
            G2_i[p,i,j] = -( -h1.real/4*(p_n)/d*k )
            if i == j:
                G2_r[p,i,j] = -const
                G2_i[p,i,j] = 0.0

In [2]:
import numpy as np
import scipy.special as scp
import cusolver
import cupy as cp

G2_r_gpu = cp.zeros_like(G2_r)
G2_i_gpu = cp.zeros_like(G2_i)
d_gpu = cp.zeros_like(d)

for p in range(sample_tri):
    for i in range(N):
        if not i in ver_num:
            r = x[p, i, :] - x[p, :, :]
            p_n = (c[p, :, :] * r).sum(axis=1)
            d = cp.sqrt(cp.square(r[:, 0]) + cp.square(r[:, 1]))

            # 在GPU上计算Hankel函数
            hankel1_imag = cp.zeros_like(d)
            hankel1_real = cp.zeros_like(d)
            cusolver.cusolverDnSbessel_J0(hankel1_imag.data.ptr, d.data.ptr, d.shape[0])
            cusolver.cusolverDnSbessel_Y0(hankel1_real.data.ptr, d.data.ptr, d.shape[0])
            
            # 将结果存储在GPU内存中
            G2_r_gpu[p, i, :] = -hankel1_imag / 4 * p_n / d * k
            G2_i_gpu[p, i, :] = -hankel1_real / 4 * p_n / d * k
            G2_r_gpu[p, i, i], G2_i_gpu[p, i, i] = -const, 0
            
            G2_r = cp.asnumpy(G2_r_gpu)
            G2_i = cp.asnumpy(G2_i_gpu)

ModuleNotFoundError: No module named 'cusolver'